# Anthropic

Anthropic is an AI safety and research company that's working to build reliable, interpretable, and steerable AI systems. Through our LiteLLM integration, you are able to easily run feedback functions with Anthropic's Claude and Claude Instant.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/anthropic_quickstart.ipynb)

In [ ]:
# ! pip install anthropic trulens_eval==0.20.3 langchain==0.0.347

In [ ]:
import os 

os.environ["ANTHROPIC_API_KEY"] = "..."

### Chat with Claude

In [ ]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

anthropic = Anthropic()

def claude_2_app(prompt):
    completion = anthropic.completions.create(
        model="claude-2",
        max_tokens_to_sample=300,
        prompt=f"{HUMAN_PROMPT} {prompt} {AI_PROMPT}",
    ).completion
    return completion

claude_2_app("How does a case reach the supreme court?")

### Initialize Feedback Function(s)

In [ ]:
from trulens.core import Tru
tru = Tru()
tru.reset_database()

In [ ]:
from trulens.external import LiteLLM
# Initialize Huggingface-based feedback function collection class:
claude_2 = LiteLLM(model_engine="claude-2")

from trulens.core import Feedback
# Define a language match feedback function using HuggingFace.
f_relevance = Feedback(claude_2.relevance).on_input_output()
# By default this will check language match on the main app input and main app
# output.

### Instrument chain for logging with TruLens

In [ ]:
from trulens.core import TruBasicApp

tru_recorder = TruBasicApp(claude_2_app,
app_id='Anthropic Claude 2',
feedbacks=[f_relevance]
)

In [ ]:
with tru_recorder as recording:
    llm_response = tru_recorder.app("How does a case make it to the supreme court?")

### Explore in a Dashboard

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(tru) # open a local streamlit app to explore

# stop_dashboard(tru) # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

### Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all